In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation

Introduce a weighting factor into the UB/UBI refinement. Effectively like adding in the contribution of the same peak over and over again.

This weighting factor could be:
- the pixel by pixel intensity ... this is just taking the center of mass position in g-vector space
- a robust function which downweights outliers ... seems non-linear or iterative for two grains (which one do you pick for the solution?)

In [ ]:
def robust_weights(UBI, gv, sigma = 0.005, eta = 0.5 ):
    """ 
    Given an approximate UBI and experimental g-vectors ...
    Computes integer hkl :        hint= VVVVVVVVVVVVVVVVVV
    Finds computed g-vectors : dot(UB), round(dot(UBI,gv))
    gerr**2 = sum_{xyz} (gobs - gcalc)**2 
    Weights are:
        Gauss term  :  exp( -gerr^2 / (2*sigma*sigma) ) ... local
        Cauchy term :  sigma*sigma / (sigma*sigma + )
    Computes p = exp()
    
    """
    hint = np.round( np.dot( UBI, gv ) )
    gcalc = np.dot( np.linalg.inv(UBI), hint )
    dg2 = ((gv - gcalc)**2).sum( axis = 0 )
    isig = 1/sigma
    # probability this peak is indexed : Gaussian errors
    pg = np.exp( - dg2*(isig*isig/2) )   # exp(0) == 1
    # probability this peak is indexed : Cauchy errors
    pe = sigma * sigma / ( sigma * sigma + dg2 )
    p = pg * eta + pe * (1-eta)
    pm = p.max()
    assert pm <= 1 and pm > 0
    return hint, p, dg2.mean() # estimate for what sigma might be

def weighted_refine(hkl, gv, weights ):
    """
    Refine an orientation matrix with weights

    Based on Paciorek et al Acta A55 543 (1999)
       UB = R H-1
    ... add a weighting per peak (like adding the peak w times)
    
    where:
       R = sum_n r_n h_n^t
       H = sum_n h_n h_n^t
       w = weights for each peak
       r = g-vectors
       h = hkl indices
    """
    R = np.zeros((3,3),float)
    H = np.zeros((3,3),float)
    assert gv.shape[0] == 3
    for w,h,g in zip(weights, hkl.T, gv.T ):
        R += (np.outer(g,h)*w)
        H += (np.outer(h,h)*w)
    HI = np.linalg.pinv(H, hermitian=True)          # might fail
    UBoptimal = np.dot(R,HI)
    UBIo = np.linalg.inv(UBoptimal)
    return UBIo, UBoptimal, R, H

In [ ]:
np.random.seed(13)
a = np.random.normal(loc=0,scale=1,size=3)
r = Rotation.from_euler('xyz', a, degrees=True )
a0 = 5.43094
ubi0 = np.eye(3)*a0
ubi1 = np.dot( ubi0, r.as_dcm() )
print(ubi0,'\n', ubi1)

In [ ]:
hi = np.mgrid[ -4:5, -4:5, -4:5 ]
hi.shape = 3,-1
ub0 = np.linalg.inv( ubi0 )
g0 = np.dot( ub0, hi )
ub1 = np.linalg.inv( ubi1 )
g1 = np.dot( ub1, hi )

In [ ]:
g0.T[3],g1.T[3]

In [ ]:
gall = np.concatenate( (g0, g1), axis=1 )
gall.shape

In [ ]:
w = np.ones( len(gall[0]) )
ubiavg = 0.5*(ubi0 + ubi1)

In [ ]:
m = (ubi0+ubi1*2)/3
e = 1e-4
for i in range(10):
    hint, w, e = robust_weights( m, gall, e, eta=0.5)
    ret = weighted_refine( hint, gall, 1-w )
    m = ret[0]
    print('e',e, 'err',((ubi0 - m)**2).sum(), ((ubi1 - m)**2).sum(), end=' ')
    ret = weighted_refine( hint, gall, w )
    m = ret[0]
    print('e',e, 'err',((ubi0 - m)**2).sum(), ((ubi1 - m)**2).sum())